In [1]:
# notebook to execute a list of jobfiles without supervision for "stream not found" or bad exit codes
# repeats same file 5 times then moves on

import os
import subprocess
import shlex
import numpy as np
from select import select
from subprocess import Popen

import astropy.time as atime

In [56]:
gps_time =  	1253326744.85 
print("  trigger_jd:", atime.Time(gps_time, format="gps").jd)

  trigger_jd: 2458750.5968038193


In [22]:

# directory where ampel job should be run (NEEDS .tmp FOLDER)
execute_directory="/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2"

# directory where jobfiles are found
jobfile_save_dir="/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/examples/calibrateKilonovaEval_jobfiles/"

#jobfiles_to_execute_file = os.listdir(jobfile_save_dir)
# list of names of jobfiles to run
jobfile_names = os.listdir(jobfile_save_dir)
#config file to use
conf_file = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/ampel_conf.yaml"

#vault file to use
vault_file = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/vault.yaml"

print(jobfile_names)


['ligo_healpix_S190408an.yaml', 'ligo_healpix_S190426c.yaml', 'ligo_healpix_S190702ah.yaml', 'ligo_healpix_S190901ap.yaml', 'ligo_healpix_S190910d.yaml', 'ligo_healpix_S190910h.yaml', 'ligo_healpix_S190923y.yaml', 'ligo_healpix_S190930t.yaml', 'ligo_healpix_S191129u.yaml', 'ligo_healpix_S191204r.yaml', 'ligo_healpix_S191205ah.yaml', 'ligo_healpix_S191213g.yaml', 'ligo_healpix_S191215w.yaml', 'ligo_healpix_S191216ap.yaml', 'ligo_healpix_S191222n.yaml', 'ligo_healpix_S200105ae.yaml', 'ligo_healpix_S200112r.yaml', 'ligo_healpix_S200115j.yaml', 'ligo_healpix_S200116ah.yaml', 'ligo_healpix_S200128d.yaml', 'ligo_healpix_S200129m.yaml', 'ligo_healpix_S200208q.yaml', 'ligo_healpix_S200213t.yaml', 'ligo_healpix_S200219ac.yaml', 'ligo_healpix_S200224ca.yaml', 'ligo_healpix_S200225q.yaml', 'ligo_healpix_S200302c.yaml', 'ligo_healpix_S200303ba.yaml', 'ligo_healpix_S200308e.yaml', 'ligo_healpix_S200311bg.yaml', 'ligo_healpix_S200316bj.yaml']


In [27]:
def execute_jobfile(job_call, execute_directory):
    command = job_call
    command_list = shlex.split(command)

    #print(command_list)

    stream_not_found_err = "Stream not found"

    os.chdir(execute_directory)

    timeout = 0.1
    retry = True
    max_retries = 5
    retry_ind = 0
    while retry:
        print("STARTING SUBPROCESS, try #", retry_ind)
        try:
            process = Popen(command_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except NameError:
            print("Error")
        retry = False

        
        while process:
            #output, error = process.communicate()
            #print(output, error)

            if process.poll() is not None:
                if process.returncode != 0:
                    print("BAD EXIT")
                    print(process.stderr.read().decode("utf-8"), end="")
                    retry = True
                    break
                print(process.stdout.read().decode("utf-8"), end="")
                process.stdout.close()
                break
            
            rlist = select([process.stdout], [], [], timeout)[0]

            for f in rlist:
                line = f.readline().decode("utf-8")
                print(line, end="")
                if line.find(stream_not_found_err) != -1:
                    print("FOUND", stream_not_found_err)
                    retry = True
                    process.kill()
                #elif line.find("Error") != -1:
                    #print("Found SyntaxError")
                    #retry = True
                    #process.kill()
                #print(f"{f.readline()}")

        

        retry_ind += 1
        if retry_ind >= max_retries:
            print("MAX RETRIES REACHED, STOPPING")
            break

        print("RETRY?", retry)

In [28]:
for jobfile_name in jobfile_names:

	jobfile = os.path.join(jobfile_save_dir, jobfile_name)

	print(f"################################################ \n################################################ \n\n\t\tNEXT JOBFILE\n\t{jobfile_name}\n\n################################################ \n################################################ \n")

	job_call = "" #f"cd {execute_directory};"
	job_call += f"ampel job --schema {jobfile} --config {conf_file} --secrets {vault_file}"

	execute_jobfile(job_call, execute_directory)

################################################ 
################################################ 

		NEXT JOBFILE
	ligo_healpix_S190408an.yaml

################################################ 
################################################ 

STARTING SUBPROCESS, try # 0
 ----------------

2023-06-29 15:23:08 CORE T3Processor:76 SHOUT
 Running ligo#0

2023-06-29 15:23:08 CORE T3Processor:119 INFO [u=T3PlainUnitExecutor]
 Processing run block 0

2023-06-29 15:23:10 CORE T3PlainUnitExecutor:76 INFO [u=HealpixTokenGenerator]
BAD EXIT

2023-06-29 15:23:04 JobCommand:323 INFO [pid=8820]
 Running job ligo

2023-06-29 15:23:05 AmpelDB:280 INFO
 Creating ligo -> stock [('localhost', 27017)]
   Creating index: {'index': (('stock', 1), ('channel', 1)), 'args': {'unique': True}}

2023-06-29 15:23:05 AmpelDB:280 INFO
 Creating ligo -> t0 [('localhost', 27017)]
   Creating index: {'index': [('id', 1)], 'args': {'unique': True}}
   Creating index: {'index': [('stock', 1)], 'args': {'sparse': Tru

KeyboardInterrupt: 